# **Introdução aos Modelos DSGE**

João Ricardo Costa Filho \\
https://www.joaocostafilho.com

# **Modelo de Ciclos de Negócio Reais (RBC) com competição imperfeita**

## Instale na máquina "virtual" o Octave e o Dynare.

In [ ]:
!sudo apt-get update
!sudo apt-get install -y x13as

In [2]:
!apt-get install octave &> /dev/null

In [3]:
!apt-get install dynare &> /dev/null

## Equilíbrio Estacionário

In [5]:
%%writefile ss.m

r = 1 / beta - 1 + delta;

steadystate = @(x)[

    % c: x(1); h: x(2); k: x(3); w: x(4);
    % y: x(5); i: x(6);

    % Famílias
    psi * x(2)^phi * x(1)^sigma - x(4);
    x(6) - delta * x(3);

    %Empresas
    x(5) - x(3)^alpha * x(2)^(1-alpha);
    r - (xi - 1) / xi * alpha * x(10) / x(8);
    x(4) - (xi - 1) / xi * ( 1 - alpha ) * x(10) / x(6);

    % Restrição de recursos
    x(5) - x(1) - x(6);

];

x = fsolve( steadystate, ones(12, 1) );

x

Overwriting ss.m


In [6]:
%%writefile RBC_SS.mod

@#define concorrenciaperfeita=0

%--------------------------------------------------------------------------------------------------------------------------------------
% 1. Definição das Variáveis
%--------------------------------------------------------------------------------------------------------------------------------------

%%% Variáveis endógenas %%%

var c  ${c}$  (long_name='Consumo')
    h  ${h}$  (long_name='Horas trabalhadas')
    k  ${k}$  (long_name='Estoque de capital')
    w  ${w}$  (long_name='Salário real')
    r  ${r}$  (long_name='Taxa de juros real')
    y  ${y}$  (long_name='PIB')
    i  ${r}$  (long_name='Investimento')
    A  ${A}$  (long_name='Produtividade Total dos Fatores')

;

%%% Variáveis exógenas %%%

varexo e ${\varepsilon_A}$   (long_name='Choque de produtividade')
;

%--------------------------------------------------------------------------------------------------------------------------------------
% 2. Calibração
%--------------------------------------------------------------------------------------------------------------------------------------

parameters phi     ${\phi}$   (long_name='Curvatura da função utilidade em relação às horas trabalhadas')
           psi     ${\psi}$   (long_name='peso da desutilidade do trabalho na função utilidade')
           sigma   ${\sigma}$ (long_name='curvatura da função utilidade')
           alpha   ${\alpha}$ (long_name='parâmetro da função de produção')
           beta    ${\beta}$  (long_name='fator de desconto')
           delta   ${\delta}$ (long_name='taxa de depreciação')
           rho     ${\rho}$   (long_name='autocorrelação da produtividade')
           xi      ${\xi}$    (long_name='elasticidade de substituição')

;


phi   = 1;
psi   = 2.29;
sigma = 2;
alpha = 0.44;
beta  = 0.97;
delta = 0.05;
rho   = 0.9;
xi    = 4;

verbatim;
run('ss.m');
end;

%--------------------------------------------------------------------------------------------------------------------------------------
% 3. Modelo
%--------------------------------------------------------------------------------------------------------------------------------------

model;

# Abar = 1;
#Uc   = c^(-sigma);
#Uc_1 = c(+1)^(-sigma);
#Uh   = -psi * h^phi;

%%%%%%%%%%%%% Famílias %%%%%%%%%%%%%

[name = 'Oferta de Trabalho']
- Uh = Uc * w;

[name = 'Equação de Euler']
Uc =  beta * ( Uc_1 * ( 1 + r(+1) - delta ) );

[name = 'Lei de Movimento do Capital']
k = ( 1 - delta ) * k(-1) + i;

%%%%%%%%%%%%% Empresas %%%%%%%%%%%%%

[name = 'Função de produção']
y = A * k(-1)^alpha * h^(1-alpha);

@#if concorrenciaperfeita==1

    [name = 'Demanda por capital']
    r = alpha * y / k;

    [name = 'Demanda por trabalho']
    w = ( 1 - alpha ) * y / h;

@#else

    [name = 'Demanda por capital']
    r = ( xi - 1 ) / xi * alpha * y / k;

    [name = 'Demanda por trabalho']
    w = ( xi - 1 ) / xi * ( 1 - alpha ) * y / h;

@#endif



%%%%%%%%%%%%% Agragação %%%%%%%%%%%%%

[name = 'Restrição de recursos']
y = c + i;

[name = 'Produtividade']
log( A ) = ( 1 - rho ) * log( Abar ) + rho * log( A(-1) ) + e;

end;

%--------------------------------------------------------------------------------------------------------------------------------------
% 4. Equilíbrio
%--------------------------------------------------------------------------------------------------------------------------------------

initval;

A   = 1;
r   = 1 / beta - 1 + delta;
c = x(1);
h = x(2);
k = x(3);
w = x(4);
y = x(5);
i = x(6);

end;

steady;
check;


Writing RBC_SS.mod


Apenas para fins didáticos:

In [7]:
%%writefile solve.m
dynare RBC_SS.mod

Writing solve.m


In [8]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 8 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 2).
Processing outputs ...
done
Preprocessing completed.

x =

   0.906440
   0.782543
   1.275532
   1.472387
   0.970217
   0.063777
   1.000000
   0.911695
   1.000000
   0.223581
   1.000000
   1.000000


STEADY-STATE RESULTS:

c 		 1.09952
h 		 0.457738
k 		 5.63177
w 		 1.26725
r 		 0.0809278
y 		 1.38111
i 		 0.281589
A 		 1

EIGENVALUES:
         Modulus             Real        Imaginary

             0.9              0.9                0
          0.9415           0.9415                0
           1.151            1.151                0
       3.832e+17       -3.832e+17                0


There are 2 eigenvalue(s) larger than 1 in modulus 
for 2 forw

## Funções Impulso-resposta

In [12]:
%%writefile RBC_CM.mod

@#include "RBC_SS.mod"

var chat
    hhat
    khat
    what
    rhat
    yhat
    ihat
    Ahat
;

model;

chat = log( c ) - log( STEADY_STATE( c ) ) * 100;

hhat = log( h ) - log( STEADY_STATE( h ) ) * 100;

khat = log( k ) - log( STEADY_STATE( k ) ) * 100;

what = log( w ) - log( STEADY_STATE( w ) ) * 100;

rhat = log( r ) - log( STEADY_STATE( y ) ) * 100;

yhat = log( y  ) - log( STEADY_STATE( y ) ) * 100;

ihat  = log( i ) - log( STEADY_STATE( i ) ) * 100;

Ahat  = log( A ) - log( STEADY_STATE( A ) ) * 100;



end;

%--------------------------------------------------------------------------------------------------------------------------------------
% 5. Simulação
%--------------------------------------------------------------------------------------------------------------------------------------

shocks;

var e;
stderr 1;
end;

stoch_simul(ar=1, order=1, irf=20, graph_format = pdf);

Overwriting RBC_CM.mod


In [13]:
%%writefile solve.m
dynare RBC_CM.mod

Overwriting solve.m


In [14]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 16 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 1).
Processing outputs ...
done
Preprocessing completed.

x =

   0.906440
   0.782543
   1.275532
   1.472387
   0.970217
   0.063777
   1.000000
   0.911695
   1.000000
   0.223581
   1.000000
   1.000000


STEADY-STATE RESULTS:

c    		 1.09952
h    		 0.457738
k    		 5.63177
w    		 1.26725
r    		 0.0809278
y    		 1.38111
i    		 0.281589
A    		 1
chat 		 -9.39289
hhat 		 77.3644
khat 		 -171.114
what 		 -23.448
rhat 		 -34.8032
yhat 		 -31.9661
ihat 		 125.463
Ahat 		 0

EIGENVALUES:
         Modulus             Real        Imaginary

             0.9              0.9                0
          0.9415           0.9415                0
           1.15